In [1]:
## load useful libraries
suppressWarnings(suppressPackageStartupMessages(library(rhdf5)))
suppressWarnings(suppressPackageStartupMessages(library(qvalue)))
suppressWarnings(suppressPackageStartupMessages(library(dplyr)))
suppressWarnings(suppressPackageStartupMessages(library(tidyverse)))
suppressWarnings(suppressPackageStartupMessages(library(ggbeeswarm)))
suppressWarnings(suppressPackageStartupMessages(library(scater)))
suppressWarnings(suppressPackageStartupMessages(library(edgeR)))
suppressWarnings(suppressPackageStartupMessages(library(SingleCellExperiment)))
suppressWarnings(suppressPackageStartupMessages(library(limma)))

In [2]:
counts_path = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_raw/10x/"
counts_suffix = "/filtered_gene_bc_matrices/1000Genomes_hs37d5/"

In [3]:
# 10X day0 cell ranger counts
runs = c("cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome",
         "cellranger201_count_22951_3_1000Genomes_hs37d5-ensembl_75_transcriptome",
         "cellranger210_count_4636STDY7138130_1000Genomes_hs37d5-ensembl_75_transcriptome", # expt40 (??), day0
         "cellranger210_count_24192_4636STDY7151182_1000Genomes_hs37d5-ensembl_75_transcriptome", # expt41, day0
         "cellranger210_count_24617_4636STDY7181831_1000Genomes_hs37d5-ensembl_75_transcriptome", # expt43, day0
         "cellranger210_count_24727_SC-RNAseq-definitive_endoderm7220110_1000Genomes_hs37d5-ensembl_75_transcriptome" # expt44, day0
        )

In [47]:
sce10x <- scater::read10XResults(
  c(
      paste0(counts_path,runs[1],counts_suffix),
      paste0(counts_path,runs[2],counts_suffix),
      paste0(counts_path,runs[3],counts_suffix),
      paste0(counts_path,runs[4],counts_suffix),
      paste0(counts_path,runs[5],counts_suffix),
      paste0(counts_path,runs[6],counts_suffix)
  )
)

In [6]:
saveRDS(sce10x, "/hps/nobackup/stegle/users/acuomo/singlecell_endodiff/sce_10x_day0.rds")

In [7]:
sce10x = readRDS("/hps/nobackup/stegle/users/acuomo/singlecell_endodiff/sce_10x_day0.rds")

In [5]:
sce10x

class: SingleCellExperiment 
dim: 20356 27452 
metadata(0):
assays(1): counts
rownames(20356): ENSG00000186092 ENSG00000237683 ... ENSG00000198695
  ENSG00000198727
rowData names(2): id symbol
colnames: NULL
colData names(2): dataset barcode
reducedDimNames(0):
spikeNames(0):

In [9]:
# table(sce10x$barcode)

In [6]:
donor_id_path = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/donor_id_10x_cardelino/"
donor_id_prefix = "donor_id_all."
donor_id_files = paste0(donor_id_path, donor_id_prefix, runs, ".csv")

In [7]:
donorfile1 = read.csv(donor_id_files[1])
dim(donorfile1)
dim(donorfile1[donorfile1$post_prob > 0.5 & donorfile1$used_in_expt == "True",])

[1] 1079470       7

[1] 2 7

In [8]:
# this could be day0, maybe just fpdm/ffdm?
d1 = donorfile1[donorfile1$post_prob > 0.9 & donorfile1$donor %in% c("ffdm"),]
c(dim(d1)[1], length(unique(d1$sample_id)), length(unique(d1$donor)))

[1] 1937 1937    1

In [9]:
donorfile2 = read.csv(donor_id_files[2])

In [10]:
# this could be day0, expt 31 (donor id seemingly confirms)
d2 = donorfile2[donorfile2$post_prob > 0.9 & donorfile2$donor %in% c("cicb","cuhk","hegp","lepk","ueah","veku"),]
c(dim(d2)[1], length(unique(d2$sample_id)), length(unique(d2$donor)))

[1] 1411 1411    6

In [11]:
donorfile3 = read.csv(donor_id_files[3])

In [12]:
# this could be day0, expt 40 (donor id seemingly confirms)
d3 = donorfile3[donorfile3$post_prob > 0.9 & donorfile3$donor %in% c("kefb","nosn","pelm","tout","zapk"),]
c(dim(d3)[1], length(unique(d3$sample_id)), length(unique(d3$donor)))

[1] 3434 3434    5

In [13]:
donorfile4 = read.csv(donor_id_files[4])

In [14]:
# this is meant to be day0, expt 41
d4 = donorfile4[donorfile4$post_prob > 0.9 & donorfile4$donor %in% c("eipl","hayt","hoik","pahc","xojn","yoch"),]
c(dim(d4)[1], length(unique(d4$sample_id)), length(unique(d4$donor)))

[1] 2911 2911    6

In [15]:
donorfile5 = read.csv(donor_id_files[5])

In [16]:
# this is meant to be day0, expt 43
d5 = donorfile5[donorfile5$post_prob > 0.9 & donorfile5$donor %in% c("aoxv","eesb","kuxp","melw","qunz","yelp"),]
c(dim(d5)[1], length(unique(d5$sample_id)), length(unique(d5$donor)))

[1] 3289 3289    6

In [17]:
donorfile6 = read.csv(donor_id_files[6])

In [18]:
# this is meant to be day0, expt 44
d6 = donorfile6[donorfile6$post_prob > 0.9 & donorfile6$donor %in% c("jejf","juuy","pipw","qehq","toco","uilk"),]
c(dim(d6)[1], length(unique(d6$sample_id)), length(unique(d6$donor)))

[1] 3962 3962    6

In [19]:
dim(sce10x)
mat = as.matrix(counts(sce10x))

[1] 20356 27452

In [20]:
rownames(mat) = paste0(rowData(sce10x)$id,"_",rowData(sce10x)$symbol)
colnames(mat) = paste0(colData(sce10x)$barcode,"_",colData(sce10x)$dataset)
head(mat[,1:10])

,AAACCTGAGCCTCGTG-1_1,AAACCTGAGGCCCTCA-1_1,AAACCTGAGTACGTTC-1_1,AAACCTGGTATGCTTG-1_1,AAACCTGGTGGGTCAA-1_1,AAACCTGTCACTTATC-1_1,AAACCTGTCGTAGATC-1_1,AAACGGGCACAGGCCT-1_1,AAACGGGCACCGATAT-1_1,AAACGGGCAGGTTTCA-1_1
ENSG00000186092_OR4F5,0,0,0,0,0,0,0,0,0,0
ENSG00000237683_AL627309.1,0,1,0,0,0,0,0,1,1,0
ENSG00000235249_OR4F29,0,0,0,0,0,0,0,0,0,0
ENSG00000185097_OR4F16,0,0,0,0,0,0,0,0,0,0
ENSG00000269831_AL669831.1,0,0,0,0,0,0,0,0,0,0
ENSG00000269308_AL645608.2,0,0,0,0,0,0,0,0,0,0


In [21]:
sample_mapping = cbind(colnames(mat), gsub("*-.*","",colnames(mat)), gsub(".*_","",colnames(mat)))
sample_mapping = as.data.frame(sample_mapping)
colnames(sample_mapping) = c("cell_id","barcode","dataset")
sample_mapping$run = runs[as.numeric(sample_mapping$dataset)]
head(sample_mapping)
length(unique(sample_mapping$cell_id))

cell_id,barcode,dataset,run
AAACCTGAGCCTCGTG-1_1,AAACCTGAGCCTCGTG,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
AAACCTGAGGCCCTCA-1_1,AAACCTGAGGCCCTCA,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
AAACCTGAGTACGTTC-1_1,AAACCTGAGTACGTTC,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
AAACCTGGTATGCTTG-1_1,AAACCTGGTATGCTTG,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
AAACCTGGTGGGTCAA-1_1,AAACCTGGTGGGTCAA,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
AAACCTGTCACTTATC-1_1,AAACCTGTCACTTATC,1,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome


[1] 27452

In [22]:
d1$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d1$sample_id))
d1$dataset = "1"
d1$cell_id = paste0(d1$barcode,"_",d1$dataset)
#
d2$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d2$sample_id))
d2$dataset = "2"
d2$cell_id = paste0(d2$barcode,"_",d2$dataset)
#
d3$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d3$sample_id))
d3$dataset = "3"
d3$cell_id = paste0(d3$barcode,"_",d3$dataset)
#
d4$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d4$sample_id))
d4$dataset = "4"
d4$cell_id = paste0(d4$barcode,"_",d4$dataset)
#
d5$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d5$sample_id))
d5$dataset = "5"
d5$cell_id = paste0(d5$barcode,"_",d5$dataset)
#
d6$barcode = gsub("10x.","",gsub(".filtered.vcf.gz","",d6$sample_id))
d6$dataset = "6"
d6$cell_id = paste0(d6$barcode,"_",d6$dataset)

In [23]:
d1$n_total_reads = NA
d1$n_alt_reads = NA
d = rbind(d1,d2,d3,d4,d5,d6)
tail(d)

,sample_id,donor,nvars_called,nvars_ovlap_hipsci,nvars_used,post_prob,used_in_expt,barcode,dataset,cell_id,n_total_reads,n_alt_reads
2812507,10x.TTTGGTTTCATTCACT-1,uilk,202,202,202,1.000000,False,TTTGGTTTCATTCACT-1,6,TTTGGTTTCATTCACT-1_6,156,44
2812908,10x.TTTGGTTTCGTTGCCT-1,pipw,113,113,113,1.000000,False,TTTGGTTTCGTTGCCT-1,6,TTTGGTTTCGTTGCCT-1_6,87,28
2814467,10x.TTTGTCACACGTCAGC-1,uilk,109,109,109,1.000000,False,TTTGTCACACGTCAGC-1,6,TTTGTCACACGTCAGC-1_6,79,26
2816222,10x.TTTGTCATCAGTTGAC-1,juuy,153,153,153,1.000000,False,TTTGTCATCAGTTGAC-1,6,TTTGTCATCAGTTGAC-1_6,111,32
2816712,10x.TTTGTCATCCCATTTA-1,juuy,153,153,153,0.999983,False,TTTGTCATCCCATTTA-1,6,TTTGTCATCCCATTTA-1_6,122,32
2817193,10x.TTTGTCATCGCACTCT-1,jejf,108,108,108,1.000000,False,TTTGTCATCGCACTCT-1,6,TTTGTCATCGCACTCT-1_6,87,23


In [38]:
d$donor_short_id[d$donor == 'ffdm'] <- 'ffdm_2'
d$donor_short_id[d$donor == 'fpdm'] <- 'fpdm_2'
d$donor_short_id[d$donor == 'lepk'] <- 'lepk_1'
d$donor_short_id[d$donor == 'cuhk'] <- 'cuhk_2'
d$donor_short_id[d$donor == 'ueah'] <- 'ueah_1'
d$donor_short_id[d$donor == 'hegp'] <- 'hegp_3'
d$donor_short_id[d$donor == 'veku'] <- 'veku_2'
d$donor_short_id[d$donor == 'cicb'] <- 'cicb_2'
d$donor_short_id[d$donor == 'kefb'] <- 'kefb_1'
d$donor_short_id[d$donor == 'nosn'] <- 'nosn_6'
d$donor_short_id[d$donor == 'pelm'] <- 'pelm_3'
d$donor_short_id[d$donor == 'tout'] <- 'tout_1'
d$donor_short_id[d$donor == 'zapk'] <- 'zapk_3'
d$donor_short_id[d$donor == 'eipl'] <- 'eipl_1'
d$donor_short_id[d$donor == 'pahc'] <- 'pahc_4'
d$donor_short_id[d$donor == 'hayt'] <- 'hayt_1'
d$donor_short_id[d$donor == 'hoik'] <- 'hoik_1'
d$donor_short_id[d$donor == 'xojn'] <- 'xojn_3'
d$donor_short_id[d$donor == 'yoch'] <- 'yoch_6'
d$donor_short_id[d$donor == 'kuxp'] <- 'kuxp_1'
d$donor_short_id[d$donor == 'yelp'] <- 'yelp_3'
d$donor_short_id[d$donor == 'aoxv'] <- 'aoxv_3'
d$donor_short_id[d$donor == 'eesb'] <- 'eesb_1'
d$donor_short_id[d$donor == 'melw'] <- 'melw_2'
d$donor_short_id[d$donor == 'qunz'] <- 'qunz_3'
d$donor_short_id[d$donor == 'qehq'] <- 'qehq_3'
d$donor_short_id[d$donor == 'jejf'] <- 'jejf_2'
d$donor_short_id[d$donor == 'juuy'] <- 'juuy_2'
d$donor_short_id[d$donor == 'uilk'] <- 'uilk_2'
d$donor_short_id[d$donor == 'pipw'] <- 'pipw_5'
d$donor_short_id[d$donor == 'toco'] <- 'toco_5'

In [39]:
head(d)

,sample_id,donor,nvars_called,nvars_ovlap_hipsci,nvars_used,post_prob,used_in_expt,barcode,dataset,cell_id,n_total_reads,n_alt_reads,donor_short_id
107,10x.AAACCTGAGCCTCGTG-1.filtered.vcf.gz,ffdm,120,120,120,0.9944113,False,AAACCTGAGCCTCGTG-1,1,AAACCTGAGCCTCGTG-1_1,NA,NA,ffdm_2
597,10x.AAACCTGAGGCCCTCA-1.filtered.vcf.gz,ffdm,203,203,203,0.9999976,False,AAACCTGAGGCCCTCA-1,1,AAACCTGAGGCCCTCA-1_1,NA,NA,ffdm_2
1087,10x.AAACCTGAGTACGTTC-1.filtered.vcf.gz,ffdm,226,226,226,1.0000000,False,AAACCTGAGTACGTTC-1,1,AAACCTGAGTACGTTC-1_1,NA,NA,ffdm_2
1577,10x.AAACCTGGTATGCTTG-1.filtered.vcf.gz,ffdm,124,124,124,1.0000000,False,AAACCTGGTATGCTTG-1,1,AAACCTGGTATGCTTG-1_1,NA,NA,ffdm_2
2067,10x.AAACCTGGTGGGTCAA-1.filtered.vcf.gz,ffdm,228,228,228,1.0000000,False,AAACCTGGTGGGTCAA-1,1,AAACCTGGTGGGTCAA-1_1,NA,NA,ffdm_2
2557,10x.AAACCTGTCACTTATC-1.filtered.vcf.gz,ffdm,212,212,212,1.0000000,False,AAACCTGTCACTTATC-1,1,AAACCTGTCACTTATC-1_1,NA,NA,ffdm_2


In [40]:
donor_long_ids = read.table("/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/GeneticRiskScores/1000G_Harmonized_CallR1/All.normalized.sample", header = T)

In [41]:
donor_long_ids$donor_short_id = gsub(".*-","",donor_long_ids$ID_2)

In [42]:
head(donor_long_ids)

ID_1,ID_2,missing,donor_short_id
0,0,0,0
NA,HPSI0216i-tony_5,0,tony_5
NA,HPSI0316i-pecz_2,0,pecz_2
NA,HPSI0216pf-tony,0,tony
NA,HPSI0316pf-zuik,0,zuik
NA,HPSI0316i-xaqm_4,0,xaqm_4


In [43]:
matches = inner_join(d, sample_mapping, by = c("dataset","cell_id"))
dim(matches)
head(matches)

[1] 16944    15

sample_id,donor,nvars_called,nvars_ovlap_hipsci,nvars_used,post_prob,used_in_expt,barcode.x,dataset,cell_id,n_total_reads,n_alt_reads,donor_short_id,barcode.y,run
10x.AAACCTGAGCCTCGTG-1.filtered.vcf.gz,ffdm,120,120,120,0.9944113,False,AAACCTGAGCCTCGTG-1,1,AAACCTGAGCCTCGTG-1_1,NA,NA,ffdm_2,AAACCTGAGCCTCGTG,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
10x.AAACCTGAGGCCCTCA-1.filtered.vcf.gz,ffdm,203,203,203,0.9999976,False,AAACCTGAGGCCCTCA-1,1,AAACCTGAGGCCCTCA-1_1,NA,NA,ffdm_2,AAACCTGAGGCCCTCA,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
10x.AAACCTGAGTACGTTC-1.filtered.vcf.gz,ffdm,226,226,226,1.0000000,False,AAACCTGAGTACGTTC-1,1,AAACCTGAGTACGTTC-1_1,NA,NA,ffdm_2,AAACCTGAGTACGTTC,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
10x.AAACCTGGTATGCTTG-1.filtered.vcf.gz,ffdm,124,124,124,1.0000000,False,AAACCTGGTATGCTTG-1,1,AAACCTGGTATGCTTG-1_1,NA,NA,ffdm_2,AAACCTGGTATGCTTG,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
10x.AAACCTGGTGGGTCAA-1.filtered.vcf.gz,ffdm,228,228,228,1.0000000,False,AAACCTGGTGGGTCAA-1,1,AAACCTGGTGGGTCAA-1_1,NA,NA,ffdm_2,AAACCTGGTGGGTCAA,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome
10x.AAACCTGTCACTTATC-1.filtered.vcf.gz,ffdm,212,212,212,1.0000000,False,AAACCTGTCACTTATC-1,1,AAACCTGTCACTTATC-1_1,NA,NA,ffdm_2,AAACCTGTCACTTATC,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome


In [45]:
try = left_join(matches, donor_long_ids[,c("ID_2", "donor_short_id")], by = "donor_short_id")
dim(try)
head(try)

[1] 16944    16

sample_id,donor,nvars_called,nvars_ovlap_hipsci,nvars_used,post_prob,used_in_expt,barcode.x,dataset,cell_id,n_total_reads,n_alt_reads,donor_short_id,barcode.y,run,ID_2
10x.AAACCTGAGCCTCGTG-1.filtered.vcf.gz,ffdm,120,120,120,0.9944113,False,AAACCTGAGCCTCGTG-1,1,AAACCTGAGCCTCGTG-1_1,NA,NA,ffdm_2,AAACCTGAGCCTCGTG,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2
10x.AAACCTGAGGCCCTCA-1.filtered.vcf.gz,ffdm,203,203,203,0.9999976,False,AAACCTGAGGCCCTCA-1,1,AAACCTGAGGCCCTCA-1_1,NA,NA,ffdm_2,AAACCTGAGGCCCTCA,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2
10x.AAACCTGAGTACGTTC-1.filtered.vcf.gz,ffdm,226,226,226,1.0000000,False,AAACCTGAGTACGTTC-1,1,AAACCTGAGTACGTTC-1_1,NA,NA,ffdm_2,AAACCTGAGTACGTTC,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2
10x.AAACCTGGTATGCTTG-1.filtered.vcf.gz,ffdm,124,124,124,1.0000000,False,AAACCTGGTATGCTTG-1,1,AAACCTGGTATGCTTG-1_1,NA,NA,ffdm_2,AAACCTGGTATGCTTG,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2
10x.AAACCTGGTGGGTCAA-1.filtered.vcf.gz,ffdm,228,228,228,1.0000000,False,AAACCTGGTGGGTCAA-1,1,AAACCTGGTGGGTCAA-1_1,NA,NA,ffdm_2,AAACCTGGTGGGTCAA,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2
10x.AAACCTGTCACTTATC-1.filtered.vcf.gz,ffdm,212,212,212,1.0000000,False,AAACCTGTCACTTATC-1,1,AAACCTGTCACTTATC-1_1,NA,NA,ffdm_2,AAACCTGTCACTTATC,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,HPSI0813i-ffdm_2


In [50]:
mat[1:5,1:5]

,AAACCTGAGCCTCGTG-1_1,AAACCTGAGGCCCTCA-1_1,AAACCTGAGTACGTTC-1_1,AAACCTGGTATGCTTG-1_1,AAACCTGGTGGGTCAA-1_1
ENSG00000186092_OR4F5,0,0,0,0,0
ENSG00000237683_AL627309.1,0,1,0,0,0
ENSG00000235249_OR4F29,0,0,0,0,0
ENSG00000185097_OR4F16,0,0,0,0,0
ENSG00000269831_AL669831.1,0,0,0,0,0


In [51]:
colnames(try)

[1] "sample_id"          "donor"              "nvars_called"      
 [4] "nvars_ovlap_hipsci" "nvars_used"         "post_prob"         
 [7] "used_in_expt"       "barcode.x"          "dataset"           
[10] "cell_id"            "n_total_reads"      "n_alt_reads"       
[13] "donor_short_id"     "barcode.y"          "run"               
[16] "ID_2"

In [53]:
sample_mapping_file <- try[,c("ID_2","barcode.x","nvars_called","nvars_used","post_prob",
                              "n_total_reads","n_alt_reads","donor_short_id","run","dataset","cell_id")]

In [55]:
colnames(sample_mapping_file)[1:2] <- c("donor_long_id","barcode")
head(sample_mapping_file)

donor_long_id,barcode,nvars_called,nvars_used,post_prob,n_total_reads,n_alt_reads,donor_short_id,run,dataset,cell_id
HPSI0813i-ffdm_2,AAACCTGAGCCTCGTG-1,120,120,0.9944113,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGAGCCTCGTG-1_1
HPSI0813i-ffdm_2,AAACCTGAGGCCCTCA-1,203,203,0.9999976,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGAGGCCCTCA-1_1
HPSI0813i-ffdm_2,AAACCTGAGTACGTTC-1,226,226,1.0000000,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGAGTACGTTC-1_1
HPSI0813i-ffdm_2,AAACCTGGTATGCTTG-1,124,124,1.0000000,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGGTATGCTTG-1_1
HPSI0813i-ffdm_2,AAACCTGGTGGGTCAA-1,228,228,1.0000000,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGGTGGGTCAA-1_1
HPSI0813i-ffdm_2,AAACCTGTCACTTATC-1,212,212,1.0000000,NA,NA,ffdm_2,cellranger201_count_22950_3_1000Genomes_hs37d5-ensembl_75_transcriptome,1,AAACCTGTCACTTATC-1_1


In [63]:
write.table(sample_mapping_file,"/hps/nobackup/stegle/users/acuomo/10x/cell_sample_map.txt", quote = F, row.names = T, col.names = T)

In [57]:
write.csv(mat,"/hps/nobackup/stegle/users/acuomo/10x/counts.csv", quote = F)